In [1]:
import os, sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import MaaSSim

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#!pip install MaaSSim
import MaaSSim.utils
import logging
from dotmap import DotMap
import MaaSSim.simulators
from MaaSSim.data_structures import structures as inData
from MaaSSim.day_to_day import S_driver_opt_out_TS, S_traveller_opt_out_TS
from MaaSSim.day_to_day import d2d_kpi_veh, d2d_kpi_pax

In [4]:
import osmnx as ox
import networkx as nx
import random
import math
import pandas as pd
import matplotlib.pyplot as plt          
import numpy as np
import seaborn as sns
from statsmodels.tsa.stattools import adfuller

In [5]:
# Competition model based on capital:
    
#     reservation wage = 12 euro/h
    
#     Level 1- Minimum Wage below Reservation Wage ---> 9.6
#     Level 2- Minimum Wage equal to Reservation Wage ---> 12
#     Level 3- Minimum Wage above the Reservation Wage ---> 14.4

Not only the remaining capital = 0 point is important, the profit>0 is also important.

SyntaxError: invalid syntax (Temp/ipykernel_32744/3350511655.py, line 9)

In [5]:
params = MaaSSim.utils.get_config('glance.json')
params.city = "Amsterdam, Netherlands"
params.paths.G = "../../data/graphs/Amsterdam.graphml"
params.paths.skim = "../../data/graphs/Amsterdam.csv"
params.vehicle_fixed_positions = True
params.times.patience = 600 #second
params.dist_threshold = 20000 #meter
params.dist_threshold_min = 3000 #meter

params.d2d.probabilistic = True
params.d2d.heterogeneous = False
params.d2d.B_income = 1
params.d2d.fuel_cost = 0.25 #euro/km

params.d2d.B_inveh_time = 1
params.d2d.B_exp_time = 2 #1.5 in oded's paper
params.PT_fare = 1 ## euro
params.PT_avg_speed = 6

params.nPM = 2 # number of platforms. platform id starts from 1
params.platforms.base_fare = 1.4 #euro
params.platforms.fare = float(1.20) #euro/km
params.platforms.min_fare = float(2) # euro
params.platforms.comm_rate = float(0.20)
params.platforms.discount = float(0.0)
params.platforms.daily_marketing = True

params.d2d.learning_d = 1 
params.d2d.B_Experience = 0.80
params.d2d.B_WOM = 0.20
params.d2d.B_Marketing = 0.0
params.d2d.diffusion_speed = 10/100 # speed for M is twice greater than WOM.
params.d2d.m = 5
params.d2d.mn = 100/12 # for NL
params.d2d.ini_att = 0.01
params.d2d.Eini_att = 0.01
params.d2d.adj_s = 2
params.VoT = 12# 10.63 # value of time per hour
params.assert_me = False
params.seed = 1
params.paths.requests = 'Amsterdam_requests.csv'
params.simTime = 4
params.d2d.res_wage = params.simTime*params.VoT #euro

In [6]:
params.threshold_u = 0.15 # if evaluation is based on capital. 0.005 if based on revenue and market share
params.max_revenue = 4000 # maximum revenue with the initial fare
params.alpha = 0.5
params.initial_capital = 50000
params.expense_per_day = 500
params.random_ini_position = False
params.step_size = 0.2 # euro/km
params.min_fare, params.max_fare  = 0.0, 3.0
params.initial_fares = (1.0,1.0)
params.min_wage_sub = True

params.nD = 1000
params.turnover_interval =50 #---------------------------------
params.nP = 2000
params.nV = 200

In [7]:
sim = MaaSSim.simulators.Try_and_Select(params=params, f_driver_out=S_driver_opt_out_TS,f_trav_out=S_traveller_opt_out_TS,
                                  kpi_veh=d2d_kpi_veh,kpi_pax=d2d_kpi_pax, logger_level=logging.WARNING)

This simulation uses albatros data
03-01-25 10:59:16-WARNING-Setting up 4h simulation at 2025-01-03 08:58:55 for 200 vehicles and 2000 passengers in Amsterdam, Netherlands
============ P1 TURN ============
03-01-25 10:59:28-WARNING-simulation time 4.5 s
Day =  0 --- on the Left cell with fare 0.8
(np1,vp1) =  (0, 0)     (np2,vp2) =  (0, 0)
--------------------------------------------
03-01-25 10:59:42-WARNING-simulation time 5.7 s
Day =  1 --- on the Left cell with fare 0.8
(np1,vp1) =  (16, 2)     (np2,vp2) =  (18, 0)
--------------------------------------------
03-01-25 10:59:58-WARNING-simulation time 6.6 s
Day =  2 --- on the Left cell with fare 0.8
(np1,vp1) =  (40, 4)     (np2,vp2) =  (34, 3)
--------------------------------------------
03-01-25 11:00:14-WARNING-simulation time 7.8 s
Day =  3 --- on the Left cell with fare 0.8
(np1,vp1) =  (50, 6)     (np2,vp2) =  (32, 5)
--------------------------------------------
03-01-25 11:00:36-WARNING-simulation time 7.9 s
Day =  4 --- on 

In [12]:
sim.competition_trajectory['P1']

[(0.8, -1.0135206799999998, 0.028125400000000002, 0.051250000000000004),
 (1.0, -0.8484261599999999, 0.02882963, 0.043625),
 (1.2, -0.7860715879999999, 0.034501583999999995, 0.045950000000000005),
 (0.8, -0.804276124, 0.035675892, 0.066925),
 (1.0, -0.6971838800000001, 0.04043179, 0.06225000000000001),
 (1.2, -0.638734588, 0.046907724, 0.060300000000000006),
 (0.8, -0.71716318, 0.038901240000000004, 0.084175),
 (1.0, -0.62122348, 0.047349014999999994, 0.07975000000000002),
 (1.2, -0.42941393600000005, 0.07144648800000002, 0.08567499999999999),
 (1.0, -0.48486431999999996, 0.06563311000000001, 0.09647499999999999),
 (1.2, -0.28642792399999994, 0.09485592200000001, 0.11479999999999999),
 (1.4, -0.31341333599999993, 0.102756213, 0.12290000000000001),
 (1.0, -0.34858855999999994, 0.084220705, 0.11574999999999999),
 (1.2, -0.18961416799999997, 0.10344706399999999, 0.1232),
 (1.4, -0.17854957200000002, 0.11779476100000003, 0.14130000000000004),
 (1.0, -0.25195818, 0.09387679, 0.1297749999999

In [11]:
params.nD = len(sim.res)

df_s = pd.DataFrame()
df_d = pd.DataFrame()

for d in range(0,params.nD):
    
    veh_exp = sim.res[d].veh_exp
    pax_exp = sim.res[d].pax_exp
    plat_exp = sim.res[d].plat_exp
    platforms = sim.res[d].platforms
    
    df_d.at[d, 'P1_EXPERIENCE_U'] = pax_exp.P1_EXPERIENCE_U.mean()
    df_d.at[d, 'P2_EXPERIENCE_U'] = pax_exp.P2_EXPERIENCE_U.mean()
    df_d.at[d, 'P1_WOM_U'] = pax_exp.P1_WOM_U.mean()
    df_d.at[d, 'P2_WOM_U'] = pax_exp.P2_WOM_U.mean()
    df_d.at[d, 'P1_MARKETING_U'] = pax_exp.P1_MARKETING_U.mean()
    df_d.at[d, 'P2_MARKETING_U'] = pax_exp.P2_MARKETING_U.mean()
    df_d.at[d, 'P1_OUT'] = pax_exp[pax_exp.platform_id==1].OUT.value_counts().get(False, 0)
    df_d.at[d, 'P2_OUT'] = pax_exp[pax_exp.platform_id==2].OUT.value_counts().get(False, 0)
    df_d.at[d, 'P1_ACTUAL_WT'] = pax_exp[pax_exp.platform_id==1].ACTUAL_WT.mean()
    df_d.at[d, 'P2_ACTUAL_WT'] = pax_exp[pax_exp.platform_id==2].ACTUAL_WT.mean()
    df_d.at[d, 'P1_MATCHING_T'] = pax_exp[pax_exp.platform_id==1].MATCHING_T.mean()
    df_d.at[d, 'P2_MATCHING_T'] = pax_exp[pax_exp.platform_id==2].MATCHING_T.mean()
    df_d.at[d, 'P1_PICKUP_T'] = veh_exp[veh_exp.platform_id==1].ARRIVES_AT_PICKUP.mean()/(veh_exp[veh_exp.platform_id==1].nRIDES.mean()*60)
    df_d.at[d, 'P2_PICKUP_T'] = veh_exp[veh_exp.platform_id==2].ARRIVES_AT_PICKUP.mean()/(veh_exp[veh_exp.platform_id==2].nRIDES.mean()*60)
    df_d.at[d, 'PICKUP_T'] = veh_exp[veh_exp.OUT==False].ARRIVES_AT_PICKUP.mean()/(veh_exp[veh_exp.OUT==False].nRIDES.mean()*60)
    
    df_d.at[d, 'P1_fare'] = plat_exp.fare[1]; df_d.at[d, 'P2_fare'] = plat_exp.fare[2]
    df_d.at[d, 'P1_revenue'] = plat_exp.revenue[1]; df_d.at[d, 'P2_revenue'] = plat_exp.revenue[2]
    df_d.at[d, 'P1_min_wage_sub'] = plat_exp.min_wage_sub[1]; df_d.at[d, 'P2_min_wage_sub'] = plat_exp.min_wage_sub[2]
    df_d.at[d, 'P1_profit'] = plat_exp.profit[1]; df_d.at[d, 'P2_profit'] = plat_exp.profit[2]
    df_d.at[d, 'P1_remaining_capital'] = plat_exp.remaining_capital[1]; df_d.at[d, 'P2_remaining_capital'] = plat_exp.remaining_capital[2]
    
    df_s.at[d, 'P1_EXPERIENCE_U'] = veh_exp.P1_EXPERIENCE_U.mean()
    df_s.at[d, 'P2_EXPERIENCE_U'] = veh_exp.P2_EXPERIENCE_U.mean()
    df_s.at[d, 'P1_WOM_U'] = veh_exp.P1_WOM_U.mean()
    df_s.at[d, 'P2_WOM_U'] = veh_exp.P2_WOM_U.mean()
    df_s.at[d, 'P1_MARKETING_U'] = veh_exp.P1_MARKETING_U.mean()
    df_s.at[d, 'P2_MARKETING_U'] = veh_exp.P2_MARKETING_U.mean()
    df_s.at[d, 'P1_OUT'] = veh_exp[veh_exp.platform_id==1].OUT.value_counts().get(False, 0)
    df_s.at[d, 'P2_OUT'] = veh_exp[veh_exp.platform_id==2].OUT.value_counts().get(False, 0)
    df_s.at[d, 'P1_IDLE_TIME'] = veh_exp[veh_exp.platform_id==1].IDLE_TIME.mean()/60
    df_s.at[d, 'P2_IDLE_TIME'] = veh_exp[veh_exp.platform_id==2].IDLE_TIME.mean()/60
    df_s.at[d, 'P1_REVENUE'] = veh_exp[veh_exp.platform_id==1].REVENUE.mean()
    df_s.at[d, 'P2_REVENUE'] = veh_exp[veh_exp.platform_id==2].REVENUE.mean()
    df_s.at[d, 'P1_ACTUAL_INC'] = veh_exp[veh_exp.platform_id==1].ACTUAL_INC.mean()
    df_s.at[d, 'P2_ACTUAL_INC'] = veh_exp[veh_exp.platform_id==2].ACTUAL_INC.mean()
    df_s.at[d, 'P1_COST'] = veh_exp[veh_exp.platform_id==1].COST.mean()
    df_s.at[d, 'P2_COST'] = veh_exp[veh_exp.platform_id==2].COST.mean()
    
    if d==0: d_cols = df_d.columns.values.tolist()
    if d==0: s_cols = df_s.columns.values.tolist()
    
    ld = sim.res[0].pax_exp.columns.values.tolist()
    ldn = [i for i in ld if i not in d_cols]
    for col in ldn:
        df_d.at[d, col] = pax_exp[pax_exp.OUT==False][col].mean()
        
    ls = sim.res[0].veh_exp.columns.values.tolist()
    lsn = [i for i in ls if i not in s_cols]
    for col in lsn:
        df_s.at[d, col] = veh_exp[veh_exp.OUT==False][col].mean()
        
run_id = '{}_1_1'.format('capital_with_low_sub')
df_d.to_csv('csv/1.1/demand_{}.csv'.format(run_id))
df_s.to_csv('csv/1.1/supply_{}.csv'.format(run_id))

In [10]:
sim.res[0]

DotMap(pax_exp=            P_U       PT_U  ACTUAL_WT  U_dif   OUT   mu   wu  nDAYS_HAILED  \
pax                                                                          
0     -5.806000 -11.141505        0.0    0.0  True  0.0  0.0           0.0   
1     -9.666667 -15.634502        0.0    0.0  True  0.0  0.0           0.0   
2    -10.217333 -16.961712        0.0    0.0  True  0.0  0.0           0.0   
3     -5.668667  -5.791260        0.0    0.0  True  0.0  0.0           0.0   
4    -15.049000 -17.108903        0.0    0.0  True  0.0  0.0           0.0   
...         ...        ...        ...    ...   ...  ...  ...           ...   
1995  -8.736333 -12.423946        0.0    0.0  True  0.0  0.0           0.0   
1996  -5.862333  -9.764037        0.0    0.0  True  0.0  0.0           0.0   
1997  -7.165000 -11.343354        0.0    0.0  True  0.0  0.0           0.0   
1998  -6.849000 -11.859975        0.0    0.0  True  0.0  0.0           0.0   
1999  -5.548667 -10.606632        0.0    0.0  Tru

In [ ]:
Useful Sources:

#NYC implements the most sever minimum wage regulation of drivers
https://www.amny.com/nyc-transit/uber-threat-layoffs-drivers-nyc-minimum-pay/?utm_source=chatgpt.com

# in response Uber is locking out drivers 
https://nypost.com/2024/10/11/business/uber-lyft-avoided-millions-in-pay-by-locking-nyc-drivers-out-of-apps-report/#:~:text=Business-,Uber%2C%20Lyft%20avoided%20millions%20in%20pay%20by%20locking,drivers%20out%20of%20apps%3A%20report&text=Uber%20and%20Lyft%20have%20been,pay%2C%20according%20to%20a%20report.

https://en.wikipedia.org/wiki/Legality_of_ridesharing_companies_by_jurisdiction?utm_source=chatgpt.com